# Exercício Python

----------------------

#### Parte 1: 

Criar uma classe **FolderMonitor** para monitorar arquivos adicionados dentro de uma pasta `src`. 
Quando um arquivo novo for adicionado, o mesmo será movido para uma pasta `dest`.

Cada pedaço do programa terá requisitos específicos a serem cumpridos.

Todas as iterações precisam apresentar mensagens informando o usuário de erros, ou operações bem sucedidas.
Utilizar o comando `input()` do python para iteragir com o usuário. 


**Pode Utilizar qualquer IDE de sua preferência**


O ideal é utilizar os conhecimentos aprendidos neste treinamento.
 * Threads
 * Encapsulamento
 * Classes em Python
 * Exceptions
 * bibliotecas python


#### Parte 2: 

O programa deverá ser transformado em um executável (único árquivo). Que deverá ser controlado por console.


#### Import some libs


In [5]:
import os
import glob
import time
import shutil
from threading import Thread

----------
Criar uma Exception Customizada caso a Pasta nao existir


In [11]:
class FolderDoesNotExist(Exception):
    
    def __init__(self, folder, msg):
        self._folder = folder 
        self._msg = msg
        
    def __str__(self):
        return f"{self._msg} : {self._folder}"
        

try:
    raise FolderDoesNotExist("teste mensagem", "teste")
except FolderDoesNotExist as e:
    print(e)

teste : teste mensagem


----------
Função para perguntar diretório da pasta à ser monitorada

Requisitos:
 * Caso a pasta não exsistir, perguntar se deseja criar a pasta ou não?
 * Mensagens para usuário

In [7]:
def _set_src_folder():
    src_ = input("Qual pasta deseja monitorar?")
    
    if not os.path.isdir(src_):
        answer_ = input("Esta pasta não existe, deseja criar? (s/n)").lower()
        while answer_ not in ["s", "n"]:
            answer_ = input("Não entendi, deseja criar a pasta? (s/n)").lower()
        if answer_ == "n":
            print("Então não tem o que fazer")
            exit(0)
        else:
            os.makedirs(src_)
            
    return src_

src = _set_src_folder()

Qual pasta deseja monitorar?src1
Esta pasta não existe, deseja criar? (s/n)s


----------
Função para perguntar o diretório da pasta destino


Requisitos:
 * Caso a pasta não exsistir, perguntar se deseja criar a pasta ou não?
 * Mensagens para usuário

In [8]:
def _set_dst_folder():
    dst_ = input("Qual destino dos arquivos?")
    while dst_ == "":
        dst_ = input("Destino inválido, qual destino dos arquivos?")
        if dst_ == "":
            continue
    if not os.path.isdir(dst_):
        print("pasta não existe, criando pasta...")
        os.makedirs(dst_)
    return dst_

dst = _set_dst_folder()

Qual destino dos arquivos?dst1
pasta não existe, criando pasta...


----------------
**Classe FolderMonitor** para realizar o monitoramento da Pasta

Requisitos:
 * Fazer o monitoramento da pasta em uma Thread.
 * Opção para alterar o valor da pasta destino dentro da classe. (@property, @property.setter)
     * Levantar Exceção caso a pasta não exista.
 

In [14]:
class FolderMonitor:
    
    def __init__(self, folder_path, dst_folder):
        
        self._folder_path = folder_path
        self._dst_folder = dst_folder
        
        self._monitor_run = True
        self._monitor_thread = Thread(target=self.monitor)
        self._monitor_thread.daemon = True
        self._monitor_thread.start()
                       
     
    def monitor(self):
        while self._monitor_run:
            files = glob.glob(os.path.join(self._folder_path, "**/*"), recursive=True)
            for src_ in files:
                print(src_)
                dst_ = os.path.join(self._dst_folder, os.path.basename(src_))
                shutil.move(src_, dst_)
            time.sleep(1)
                
        
    def stop(self):
        self._monitor_run = False
        self._monitor_thread.join()
        print('End!!')
        
    @property
    def dst_folder(self):
        return self._dst_folder
    
    
    @dst_folder.setter
    def dst_folder(self, new_path):
        if not os.path.isdir(new_path):
                raise FolderDoesNotExist("Caminho da pasta não existe", new_path)
        self._dst_folder = new_path

folder_monitor = FolderMonitor(src, dst)

----------
**Monitorando**

Requisitos:
   * Usuário pode utilizar o comando `sair` para encerrar o monitoramento
   * Usuário pode utilizar um comando para alterar o caminho da pasta destino em durante à execução

In [15]:
print("monitorando...")
while 1:
    answer = input()
    if answer == 'sair':
        print("Saindo...")
        break
    
    if answer.find('dst:') == 0:
        new_path = answer.split(":")[1]
        try:
            folder_monitor.dst_folder = new_path
        except FolderDoesNotExist:            
            os.makedirs(new_path)
            
            

monitorando...
dst:dst2
src1\art.jpg
sair
Saindo...
